In [ ]:
!python --version
!pip install --upgrade pip
!pip install 'keras<3.0.0' mediapipe-model-maker

Python 3.10.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import text_classifier

import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
data_path = "/content/cleanedTrainingLabeled.csv"
data_dir = os.path.join(os.path.dirname(data_path), 'cleanedTrainingLabeled')  # folder name

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/cleanedTrainingLabeled.csv')

In [ ]:
# Filter out rows where the Label column contains "Neutral"
df_filtered = df[~df['Label'].str.contains('Neutral')]

# Save the filtered DataFrame back to a CSV file
df_filtered.to_csv('filtered_file.csv', index=False)

df_valid = pd.read_csv('/content/cleanedValidatingLabeled.csv')

df_valid_filtered = df_valid[~df['Label'].str.contains('Neutral')]

df_valid_filtered.to_csv('df_valid_filtered.csv', index=False)

<ipython-input-5-438d6ab81dda>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_valid_filtered = df_valid[~df['Label'].str.contains('Neutral')]


In [ ]:
csv_params = text_classifier.CSVParams(
    text_column='Lyrics', label_column='Label')#, delimiter='\n')
train_data = text_classifier.Dataset.from_csv(
    filename=os.path.join(os.path.join("/content/filtered_file.csv")),
    csv_params=csv_params)
validation_data = text_classifier.Dataset.from_csv(
    filename=os.path.join(os.path.join("/content/df_valid_filtered.csv")),
    csv_params=csv_params)

In [ ]:
supported_model = text_classifier.SupportedModels.MOBILEBERT_CLASSIFIER
hparams = text_classifier.BertHParams(epochs=10, batch_size=32, learning_rate=2e-5, export_dir="bert_exported_models")
options = text_classifier.TextClassifierOptions(supported_model=supported_model, hparams=hparams)

In [ ]:
bert_model = text_classifier.TextClassifier.create(train_data, validation_data, options)

Using existing cache files at /tmp/tmpfk_swjn5/4d10cd6800371751066ebc50902236e5
Using existing cache files at /tmp/tmp7l4hlb9l/59f99631e2aecfe8372dc3641cfb5b48
Epoch 1/10
19/19 [==============================] - 397s 13s/step - loss: 1.5745 - accuracy: 0.5115 - precision: 0.6049 - recall: 0.6475 - val_loss: 0.3274 - val_accuracy: 0.5517 - val_precision: 0.6557 - val_recall: 0.6897
Epoch 2/10
19/19 [==============================] - 233s 12s/step - loss: 0.2945 - accuracy: 0.5526 - precision: 0.6412 - recall: 0.6580 - val_loss: 0.2026 - val_accuracy: 0.6322 - val_precision: 0.6548 - val_recall: 0.9483
Epoch 3/10
19/19 [==============================] - 245s 13s/step - loss: 0.2030 - accuracy: 0.5674 - precision: 0.6422 - recall: 0.7076 - val_loss: 0.1701 - val_accuracy: 0.6667 - val_precision: 0.6667 - val_recall: 1.0000
Epoch 4/10
19/19 [==============================] - 230s 12s/step - loss: 0.1791 - accuracy: 0.6234 - precision: 0.6742 - recall: 0.7781 - val_loss: 0.1712 - val_accura

In [ ]:
metrics = bert_model.evaluate(validation_data)
print(f'Test loss:{metrics[0]}, Test accuracy:{metrics[1]}')

Using existing cache files at /tmp/tmp7l4hlb9l/59f99631e2aecfe8372dc3641cfb5b48
3/3 [==============================] - 10s 3s/step - loss: 0.1818 - accuracy: 0.6322 - precision: 0.6857 - recall: 0.8276
Test loss:0.18181534111499786, Test accuracy:0.6321839094161987


In [ ]:
bert_model.export_model()

In [ ]:
!wget -O "/content/model.tflite"

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [ ]:
INPUT_TEXT = "I'm looking forward to what will come next."

In [ ]:
from mediapipe.tasks import python
from mediapipe.tasks.python import text

# STEP 2: Create an TextClassifier object.
base_options = python.BaseOptions(model_asset_path="/content/model.tflite")
options = text.TextClassifierOptions(base_options=base_options)
classifier = text.TextClassifier.create_from_options(options)

# STEP 3: Classify the input text.
classification_result = classifier.classify(INPUT_TEXT)

# STEP 4: Process the classification result. In this case, print out the most likely category.
top_category = classification_result.classifications[0].categories[0]
print(f'{top_category.category_name} ({top_category.score:.2f})')

In [ ]:
bert_model.export_model()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = df_filtered
# Extract the first lyric entry
first_lyric = df.iloc[1]["Lyrics"]
print(first_lyric)

Quarter in a payphone
Drying laundry on the line
Watching sun tea in the window
Pocket watch tellin' time
Seems like only yesterday, I'd get a blank cassette
Record the country countdown 'cause I couldn't buy it yet
If we drove all the way to Dallas just to buy an Easter dress
We'd take along a Rand McNally, stand in line to pay for gas
God knows that shifting gears ain't what it used to be
I learned to drive that '55, just like a queen, three on a tree
Hey, what ever happened to waiting your turn
Doing it all by hand?
'Cause when everything is handed to you
It's only worth as much as the time put in
It all just seems so good the way we had it
Back before everything became automatic
If you had something to say
You'd write it on a piece of paper
Then you put a stamp on it
And they'd get it three days later
Boys would call the girls
And girls would turn them down
Staying married was the only way to work your problems out
Hey, what ever happened to waiting your turn
Doing it all by hand?


In [ ]:
# processed_lyric = [{'Lyrics': first_lyric}]
# # Make predictions
# prediction = bert_model.predict_single([first_lyric])

# # Print or use prediction as needed
# print(prediction)

# preprocessed_lyrics = first_lyric.lower().replace(",", "").split()

# # Make a prediction
# predictions = bert_model.predict(preprocessed_lyrics)

AttributeError: '_BertClassifier' object has no attribute 'predict'

In [ ]:
# Preprocess your lyrics (replace this with your actual preprocessing steps)
# lyrics = "I'm feeling happy today"

# first_lyric_second = df.iloc[1]
# # preprocessed_lyrics = lyrics.lower().replace(",", "").split()

# single_text_data = text_classifier.Dataset.from_csv([first_lyric_second], csv_params= csv_params)


# # Make a prediction
# predictions = bert_model.evaluate(single_text_data)

# # Access the predicted class and probability score
# predicted_class = predictions[0][0]  # Index 0 for class, 1 for probability
# predicted_score = predictions[0][1]  # Index 1 for probability

# print(f"Predicted Class: {predicted_class}")
# print(f"Predicted Score: {predicted_score:.2f}")  # Format score to 2